Short Backwarding for Selecting the Best Historical Agent in a Consulting System for Portfolio Allocation with Deep Reinforcement Learning

## 5.0 Feature Reduction
---

Autoencoders are used to reduce the number of technical indicators to be represented by only two features

### 5.1 Import Relevant Libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from numpy import array
from keras.models import Model, Sequential
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model
from sklearn import preprocessing

### 5.2 Load the Data

In [2]:
df = pd.read_csv('./df.csv')
%store -r df

In [3]:
df.head()

,date,tic,close,high,low,open,volume,atr,bbw,obv,cmf,macd,adx,sma,ema,cci,rsi,cov_list
0,2009-03-20,AXP,9.658469,13.190000,12.120000,13.190000,31088200.0,20.045677,230.002536,-5.563923e+10,-19.695017,-0.890151,4.694125,18.653711,19.068885,-76.648004,47.215954,"[[0.0026107181410731633, 0.0012702991646503876..."
1,2009-03-20,CAT,17.987209,28.900000,26.730000,28.629999,16531300.0,19.965263,230.668541,-5.562269e+10,-19.455215,-0.910501,4.391543,17.160566,18.902473,-21.133828,49.482302,"[[0.0026107181410731633, 0.0012702991646503876..."
2,2009-03-20,CSCO,10.789354,16.570000,15.750000,16.370001,66078200.0,18.192457,230.960632,-5.568877e+10,-18.941235,-1.490257,4.276725,16.178579,17.654301,-64.686580,47.580937,"[[0.0026107181410731633, 0.0012702991646503876..."
3,2009-03-20,CVX,35.435562,67.980003,64.269997,67.540001,23811700.0,22.092276,230.081354,-5.566496e+10,-18.625545,0.038581,4.799891,17.896791,20.389880,114.564514,54.086535,"[[0.0026107181410731633, 0.0012702991646503876..."
4,2009-03-20,DIS,14.977372,17.980000,17.080000,17.799999,17766600.0,21.718605,230.298636,-5.568273e+10,-18.589908,-0.396044,4.597209,18.082995,19.557186,-54.702746,48.685249,"[[0.0026107181410731633, 0.0012702991646503876..."


In [4]:
# Get the list of all the features
features_list = list(df.columns)[7:-1]

In [5]:
print(features_list)

['atr', 'bbw', 'obv', 'cmf', 'macd', 'adx', 'sma', 'ema', 'cci', 'rsi']


### 5.2 Construct a Data Frame of Features

In [6]:
# construct a data frame of features
features_df = df[features_list]
features_df.index = df['date']

In [7]:
features_df.head()

,atr,bbw,obv,cmf,macd,adx,sma,ema,cci,rsi
date,,,,,,,,,,
2009-03-20,20.045677,230.002536,-5.563923e+10,-19.695017,-0.890151,4.694125,18.653711,19.068885,-76.648004,47.215954
2009-03-20,19.965263,230.668541,-5.562269e+10,-19.455215,-0.910501,4.391543,17.160566,18.902473,-21.133828,49.482302
2009-03-20,18.192457,230.960632,-5.568877e+10,-18.941235,-1.490257,4.276725,16.178579,17.654301,-64.686580,47.580937
2009-03-20,22.092276,230.081354,-5.566496e+10,-18.625545,0.038581,4.799891,17.896791,20.389880,114.564514,54.086535
2009-03-20,21.718605,230.298636,-5.568273e+10,-18.589908,-0.396044,4.597209,18.082995,19.557186,-54.702746,48.685249


In [8]:
features_df.shape

(75640, 10)

In [9]:
df.shape

(75640, 18)

### 5.3 Normalize the Features and Construct an Autoencoder

In [10]:
features_array = np.array(features_df)

features_scaler = preprocessing.MinMaxScaler()
features_normalised = features_scaler.fit_transform(features_array)

In [11]:
features_normalised.shape

(75640, 10)

In [12]:
# rescale the features array
features_normalised = features_normalised.reshape(-1,20,10)

In [13]:
features_normalised.shape

(3782, 20, 10)

In [14]:
# define model
model = Sequential()
model.add(LSTM(4, activation='relu', input_shape=(20,10)))
model.add(RepeatVector(20))
model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(10)))
model.compile(optimizer='adam', loss='mse')

In [15]:
# fit model
model.fit(features_normalised, features_normalised, epochs=200, verbose=1)
#plot_model(model, show_shapes=True, to_file='./results/reconstruct_lstm_autoencoder.png')

Epoch 1/200

119/119 [==============================] - 5s 16ms/step - loss: 0.0531
Epoch 2/200
119/119 [==============================] - 2s 15ms/step - loss: 0.0117
Epoch 3/200
119/119 [==============================] - 2s 15ms/step - loss: 0.0087
Epoch 4/200
119/119 [==============================] - 2s 15ms/step - loss: 0.0076
Epoch 5/200
119/119 [==============================] - 2s 16ms/step - loss: 0.0065
Epoch 6/200
119/119 [==============================] - 2s 16ms/step - loss: 0.0060
Epoch 7/200
119/119 [==============================] - 2s 16ms/step - loss: 0.0056
Epoch 8/200
119/119 [==============================] - 2s 16ms/step - loss: 0.0051
Epoch 9/200
119/119 [==============================] - 2s 15ms/step - loss: 0.0046
Epoch 10/200
119/119 [==============================] - 2s 15ms/step - loss: 0.0041
Epoch 11/200
119/119 [==============================] - 2s 15ms/step - loss: 0.0038
Epoch 12/200
119/119 [==============================] - 2s 16ms/step - loss: 0.0035


In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 4)                 240       
                                                                 
 repeat_vector (RepeatVecto  (None, 20, 4)             0         
 r)                                                              
                                                                 
 lstm_1 (LSTM)               (None, 20, 50)            11000     
                                                                 
 lstm_2 (LSTM)               (None, 20, 100)           60400     
                                                                 
 time_distributed (TimeDist  (None, 20, 10)            1010      
 ributed)                                                        
                                                                 
Total params: 72650 (283.79 KB)
Trainable params: 72650 

In [17]:
# connect the encoder LSTM as the output layer
model_feature = Model(inputs=model.inputs, outputs=model.layers[1].output)
#plot_model(model_feature, show_shapes=True, show_layer_names=True, to_file='./results/lstm_encoder.png')

In [18]:
model_feature.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_input (InputLayer)     [(None, 20, 10)]          0         
                                                                 
 lstm (LSTM)                 (None, 4)                 240       
                                                                 
 repeat_vector (RepeatVecto  (None, 20, 4)             0         
 r)                                                              
                                                                 
Total params: 240 (960.00 Byte)
Trainable params: 240 (960.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### 5.4 Get the Reconstructed Features

In [19]:
# get the feature vector for the input sequence
yhat = model_feature.predict(features_normalised)
print(yhat.shape)

119/119 [==============================] - 0s 1ms/step
(3782, 20, 4)


In [20]:
# reshape the vector
features_reduced = yhat.reshape(-1,4)

In [21]:
df.shape

(75640, 18)

In [22]:
features_reduced.shape

(75640, 4)

In [23]:
# Copy original data frame and drop the original features
df_reduced = df.copy()
df_reduced = df_reduced .drop(features_list, axis=1)

In [24]:
df_reduced.head()

,date,tic,close,high,low,open,volume,cov_list
0,2009-03-20,AXP,9.658469,13.190000,12.120000,13.190000,31088200.0,"[[0.0026107181410731633, 0.0012702991646503876..."
1,2009-03-20,CAT,17.987209,28.900000,26.730000,28.629999,16531300.0,"[[0.0026107181410731633, 0.0012702991646503876..."
2,2009-03-20,CSCO,10.789354,16.570000,15.750000,16.370001,66078200.0,"[[0.0026107181410731633, 0.0012702991646503876..."
3,2009-03-20,CVX,35.435562,67.980003,64.269997,67.540001,23811700.0,"[[0.0026107181410731633, 0.0012702991646503876..."
4,2009-03-20,DIS,14.977372,17.980000,17.080000,17.799999,17766600.0,"[[0.0026107181410731633, 0.0012702991646503876..."


In [25]:
# convert the reduced features to a data frame and merge with the original data frame
features_reduced_df = pd.DataFrame(features_reduced, columns=['f01','f02','f03','f04'])

In [26]:
features_reduced_df.head()

,f01,f02,f03,f04
0,0.218444,1.06041,5.250239,1.707034
1,0.218444,1.06041,5.250239,1.707034
2,0.218444,1.06041,5.250239,1.707034
3,0.218444,1.06041,5.250239,1.707034
4,0.218444,1.06041,5.250239,1.707034


In [27]:
df_reduced[['f01','f02','f03','f04']] = features_reduced_df[['f01','f02','f03','f04']]

In [28]:
df_reduced.head()

,date,tic,close,high,low,open,volume,cov_list,f01,f02,f03,f04
0,2009-03-20,AXP,9.658469,13.190000,12.120000,13.190000,31088200.0,"[[0.0026107181410731633, 0.0012702991646503876...",0.218444,1.06041,5.250239,1.707034
1,2009-03-20,CAT,17.987209,28.900000,26.730000,28.629999,16531300.0,"[[0.0026107181410731633, 0.0012702991646503876...",0.218444,1.06041,5.250239,1.707034
2,2009-03-20,CSCO,10.789354,16.570000,15.750000,16.370001,66078200.0,"[[0.0026107181410731633, 0.0012702991646503876...",0.218444,1.06041,5.250239,1.707034
3,2009-03-20,CVX,35.435562,67.980003,64.269997,67.540001,23811700.0,"[[0.0026107181410731633, 0.0012702991646503876...",0.218444,1.06041,5.250239,1.707034
4,2009-03-20,DIS,14.977372,17.980000,17.080000,17.799999,17766600.0,"[[0.0026107181410731633, 0.0012702991646503876...",0.218444,1.06041,5.250239,1.707034


In [29]:
data_df = df_reduced.copy()

In [30]:
%store data_df
data_df.to_csv('./data_df.csv')


Stored 'data_df' (DataFrame)
